#Hard coded values
Column names
'createdatl,'updatedatl','marketproduct_id','Eddress ID','product_barcode','marketproduct_name_de','segment_name',
'brand_name','category_name_en','subcategory_name_en','supplier_names','list_price,'g10_price','transfer_groups','isvisible'

Segment filter
"Asset", "Delist", "None", "Test", "Only Promo"

category managers
"Marius","Fabian","Gina","Hamid","Amadeus","Rozan","Francisco","Eike","Evrim","Leonard","Andrea"

Cities
"Berlin","Berlin Mini", "Dresden","Leipzig","Hamburg","Bremen","Hannover","Düsseldorf","Dortmund","Köln","Bonn","Essen","München", "Nürnberg/Fürth","Frankfurt","Offenbach","Stuttgart","Heidelberg","Mannheim","Karlsruhe","Darmstadt","Augsburg"

Status_update_exception_Dictionary = {\
"PUBLISHED":['UNPUBLISHED - New Ingoing'],
"PUBLISHED - Outgoing":['PUBLISHED - Deplete']
}


#Using Instructions
1 - save panel report on desktop and rename until date. (ex - G10andGetirBuyuk_AllProducts_without_TRDE_20230605.xlsx)
2 - save eddress export all on desktop and rename as Inventory.xls
3 - run the code - you will have a sheet on your desktop "Status_update"





In [3]:
#import packages
import pandas as pd
import numpy as np
from datetime import datetime as dt
today = dt.now()

#Read the downloaded report
Path_A = "/Users/swaroopbasavarajmusti/Desktop/G10andGetirBuyuk_AllProducts_without_TRDE.xlsx"

Assortment = pd.read_excel(Path_A)

#change the marketplace product id column name (not required but to avoid confusion)

Assortment.rename(columns={'marketproduct_id':'Getir Product Id'}, inplace=True)

#select only required columns

Assortment = Assortment[['createdatl','updatedatl','Getir Product Id','product_barcode','marketproduct_name_de','segment_name',
'brand_name','category_name_en','subcategory_name_en','supplier_names','net_net_buying_price_with_vat','g10_price','transfer_groups','isvisible']]

#filter unwanted segments from the dataframe
Segment_filters = ["Asset", "None", "Test"]
Assortment = Assortment[~Assortment.segment_name.isin(Segment_filters)].reset_index(drop = True)

#sort the values based on the date created
Assortment = Assortment.sort_values(by="createdatl",ascending = False).reset_index(drop = True)

#------------------------------

#creating dictionary to match the product ids

Path_B = "/Users/swaroopbasavarajmusti/Desktop/Inventory.xls"
Dictionary = pd.read_excel(Path_B)

Dictionary = Dictionary[Dictionary['Getir Product Id'].notna()].reset_index(drop = True)
Dictionary = Dictionary[['Getir Product Id','Product Id','Product SKU','Enabled / Disabled']]


#------------------------------

#left joining Panel assortment with eddress ID dictionary

Assortment_Inventory = pd.merge(Assortment, 
                      Dictionary, 
                      on ='Getir Product Id', 
                      how ='left')

#------------------------------

# read purchase order report 

Path_C = "/Users/swaroopbasavarajmusti/Desktop/G10_and_GB_purchaseorderInfo.xlsx"
PO = pd.read_excel(Path_C)

#rename primary keys and reduce to only required columns
PO.rename(columns={'Item':'Getir Product Id'}, inplace=True)
PO.rename(columns={'Job Status Name':'Job_status'}, inplace=True)

#filter unwanted columns from the dataframe and create only required dataframe
PO = PO[['Getir Product Id','Purchaseorder Createdat L','Job_status']]

#filter unwanted job status from the dataframe and create only required dataframe
Job_Status_Name_filters = ["CANCELLED"]
PO = PO[~PO.Job_status.isin(Job_Status_Name_filters)].reset_index(drop = True)

#look for unique values of products ordered and create ordered status
Ordered_products = pd.DataFrame(data=PO['Getir Product Id'].unique(), columns=["Getir Product Id"])
Ordered_products['Order_Status'] = 'TRUE'

#------------------------------

#join the two table with primary
Assortment_Joined = pd.merge(Assortment_Inventory, 
                      Ordered_products, 
                      on ='Getir Product Id', 
                      how ='left')

#------------------------------

#filtering only required columns
Assortment_Joined = Assortment_Joined[['createdatl','updatedatl','Getir Product Id','Product Id','Product SKU','product_barcode','marketproduct_name_de','segment_name',
'brand_name','category_name_en','subcategory_name_en','supplier_names','net_net_buying_price_with_vat','g10_price','isvisible','transfer_groups','Enabled / Disabled','Order_Status']]

#convert datetime format to just date
Assortment_Joined['createdatl'] = Assortment_Joined['createdatl'].dt.date
Assortment_Joined['updatedatl'] = Assortment_Joined['updatedatl'].dt.date

#------------------------------

# #create categories and categories manager dictionary
# Category_dictionary = {\
# "Marius":['Baby','Beer','Home Care','Personal Care','Pet','Spirits & Co.'],
# "Fabian":['Bakery'],
# "Gina":['Breakfast','Pantry','Pasta & Rice'],
# "Hamid":['Cheese & Co.','Milk & Eggs','Yoghurt & Co.','Dairy','Cheese & Deli','Vegan & Veggie'],
# "Amadeus":['Christmas','Confectionery','Crisps & Snacks','Easter'],
# "Francisco":['Cold Cuts','Fresh & Ready','Meat & Co.'],
# "Eike":['Drinks','Coffee & Tea'],
# "Evrim":['Frozen Food','Ice Cream'],
# "Leonard":['Fruit & Veg'],
# "Andrea":['Wine & Co.'],
# "Other":['DUMMY TEST KATEGORI GE',"Mother's Day",'Top Deals','Earthquake Aid','']}

# #------------------------------

# #match the categories and get the name of category manager
# for ind, row in Assortment_Joined.iterrows():
#     if Assortment_Joined.loc[ind, 'category_name_en'] == "":
#         Assortment_Joined.loc[ind, 'Buyer'] = "Not Assigned"
#     else:
#         Category = Assortment_Joined.loc[ind, 'category_name_en']
#         Assortment_Joined.loc[ind, 'Buyer'] = [k for k, v in Category_dictionary.items() if Category in v]


    
# #get only the name without brackets
# Assortment_Joined['Buyer'] = Assortment_Joined['Buyer'].astype(str)
# Assortment_Joined['Buyer'] = Assortment_Joined['Buyer'].str[2:-2]

#------------------------------

#change the eddress enabled/disabled status to true and false
for ind,row in Assortment_Joined.iterrows():
    if Assortment_Joined.loc[ind, 'Enabled / Disabled'] == 'Enabled':
        Assortment_Joined.loc[ind, 'Live Gorillas app'] = "TRUE"
    else:
        Assortment_Joined.loc[ind, 'Live Gorillas app'] = "FALSE"

        
#------------------------------

#Generate the y/n combination for transfer group(mfc) checks
Assortment_Joined['transfer_groups'] = Assortment_Joined['transfer_groups'].astype(str)

#create a list of cities 
cities = ["Berlin","Berlin Mini", "Dresden","Leipzig","Hamburg","Bremen",\
          "Hannover","Düsseldorf","Dortmund","Köln","Bonn","Essen","München",\
          "Nürnberg/Fürth","Frankfurt","Offenbach","Stuttgart","Heidelberg",\
          "Mannheim","Karlsruhe","Darmstadt","Augsburg"]

#iterate over transfer groups to check for cities
for c in cities:
    for ind, row in Assortment_Joined.iterrows():
        x = "merged "+c
        if c in Assortment_Joined.loc[ind, 'transfer_groups']:
            Assortment_Joined.loc[ind, c] = "Y"
        else:
            Assortment_Joined.loc[ind, c] = "N"

#------------------------------

#renaming and del column
Assortment_Joined.rename(columns={'createdatl':'Product Card Creation Date'}, inplace=True)
Assortment_Joined.rename(columns={'updatedatl':'Delisting date (TBD)'}, inplace=True)
Assortment_Joined.rename(columns={'net_net_buying_price_with_vat':'Buying Price (w VAT)'}, inplace=True)
Assortment_Joined.rename(columns={'g10_price':'Selling Price (w VAT)'}, inplace=True)
Assortment_Joined.rename(columns={'isvisible':'Live Getir app'}, inplace=True)
Assortment_Joined.rename(columns={'Product Id':'Eddress Product Id'}, inplace=True)
Assortment_Joined.rename(columns={'product_barcode':'GTIN_Panel'}, inplace=True)
Assortment_Joined.rename(columns={'Product SKU':'GTIN_Eddress'}, inplace=True)
Assortment_Joined.rename(columns={'segment_name':'Segment_Panel'}, inplace=True)

#------------------------------
#additional formatting
Assortment_Joined['Delisting date (TBD)'] = ''
del Assortment_Joined['transfer_groups']
Assortment_Joined['Order_Status'] = Assortment_Joined['Order_Status'].fillna('FALSE')
del Assortment_Joined['Enabled / Disabled']
        
#------------------------------

#sorting and reset index
Assortment_Joined.sort_values(by='Product Card Creation Date', ascending = False ,inplace = True)
Assortment_Joined = Assortment_Joined.reset_index(drop=True)

#-------------------------------

#adding mdm data to dataset to get the product status on mdm

Path_D = "/Users/swaroopbasavarajmusti/Desktop/mdm_data.xlsm"
mdm_data_all = pd.read_excel(Path_D,'Entities')
mdm_data_all = mdm_data_all.rename(columns=mdm_data_all.iloc[0])
mdm_data_all = mdm_data_all.iloc[1:].reset_index(drop = True)
mdm_data = mdm_data_all
mdm_data = mdm_data[['Eddress Product ID','Web Pages.File Name','Product Status']]
mdm_data = mdm_data[mdm_data['Eddress Product ID'].notna()].reset_index(drop =True)
mdm_data.rename(columns={'Eddress Product ID':'Eddress Product Id'}, inplace=True)
mdm_data.rename(columns={'Product Status':'Segment_MDM'}, inplace=True)

Merged_assortment = pd.merge(Assortment_Joined, 
                      mdm_data, 
                      on ='Eddress Product Id', 
                      how ='left')

#-------------------------------
#checking if the product is created on eddress by creating a column

for ind,row in Merged_assortment.iterrows():
    if pd.isnull(Merged_assortment.loc[ind, 'Eddress Product Id']) == True:
        Merged_assortment.loc[ind, 'Created on Eddress'] = "FALSE"
    else:
        Merged_assortment.loc[ind, 'Created on Eddress'] = "TRUE"

#-------------------------------

#create segment dictionary and segment attributes to sync statuses
Segment_Dictionary = {\

"PUBLISHED":['Regular','Regional'],
"UNPUBLISHED - New Ingoing":['Launch'],
"PUBLISHED - Deplete":['Seasonal','Only Promo','Delist On Depletion','Consumable'],
"UNAVAILABLE - OOS Supplier":['Stand By','Supply Shortage'],
"UNPUBLISHED - Delisted":['Delist','None'] 
}

#create new segment statuts based on dictionary
for ind, row in Merged_assortment.iterrows():
    if pd.isnull(Merged_assortment.loc[ind, 'Segment_Panel']) == True:
        Merged_assortment.loc[ind, 'Segment_New'] = ""
    else:
        segment = str(Merged_assortment.loc[ind, 'Segment_Panel'])
        Merged_assortment.loc[ind, 'Segment_New'] = [k for k, v in Segment_Dictionary.items() if segment in v]

#get only the Segments without brackets
Merged_assortment['Segment_New'] = Merged_assortment['Segment_New'].astype(str)
Merged_assortment['Segment_New'] = Merged_assortment['Segment_New'].str[2:-2]

#create a filter column to filter products that have to be updated
for ind, row in Merged_assortment.iterrows():
    if pd.isnull(Merged_assortment.loc[ind, 'Segment_MDM']) == True:
        Merged_assortment.loc[ind, 'Segment_update'] = ""
    elif Merged_assortment.loc[ind, 'Segment_New'] == Merged_assortment.loc[ind, 'Segment_MDM']:
        Merged_assortment.loc[ind, 'Segment_update'] = False
    else:
        Merged_assortment.loc[ind, 'Segment_update'] = True

#-------------------------------


#organising the columns based on requirements
Merged_assortment = Merged_assortment[[\
#Update dates
'Product Card Creation Date',\
'Delisting date (TBD)',\

#ids
'Getir Product Id',\
'Eddress Product Id',\

#segments
'Created on Eddress',\
'Segment_MDM',\
'Segment_Panel',\

#product statusus on multiple tools
'Live Getir app',\
'Live Gorillas app',\

#gtins details
'GTIN_Eddress',\
'GTIN_Panel',\
'marketproduct_name_de',\
'brand_name',\
'category_name_en',\
'subcategory_name_en',\
'supplier_names',\
'Buying Price (w VAT)',\
'Selling Price (w VAT)',\
#'Buyer',\


#MFCs
'Berlin','Berlin Mini','Dresden','Leipzig','Hamburg','Bremen','Hannover','Düsseldorf','Dortmund','Köln','Bonn','Essen','München','Nürnberg/Fürth','Frankfurt','Offenbach','Stuttgart','Heidelberg','Mannheim','Karlsruhe','Darmstadt','Augsburg',\

#filters
'Order_Status',\
'Segment_New',\
'Segment_update'
    ]]


#-------------------------------
# Get the recent updates and sync statuses, drop exceptions and create MDM uploadable forrmat
#on-hold - eddress translation

New_status_df = Merged_assortment[['Getir Product Id','Eddress Product Id','Segment_MDM','Segment_New','Segment_update']]
New_status_df = New_status_df.loc[New_status_df['Segment_update'] == True]

New_status_df = New_status_df[['Getir Product Id','Eddress Product Id','Segment_MDM','Segment_New','Segment_update']].reset_index(drop = True)

New_status_df = New_status_df.drop(New_status_df[(New_status_df['Segment_New'] == 'UNPUBLISHED - New Ingoing') & \
                (New_status_df['Segment_MDM'] == 'PUBLISHED')].index)

New_status_df = New_status_df.drop(New_status_df[(New_status_df['Segment_New'] == 'PUBLISHED - Deplete') & \
                (New_status_df['Segment_MDM'] == 'PUBLISHED - Outgoing')].index).reset_index(drop = True)

#-------------------------------
#merge mdm data format with new statuses and create a template format dataframe

MDM_df = mdm_data_all[['Action','Type','ID','Name','Buying Categories','GS1 Taxonomy','GIPID',\
                                        'Trade Item Identification: GTIN','Colli','Netsuite Vendor ID',\
                                         'Eddress Product ID']]
MDM_df = MDM_df[MDM_df['Eddress Product ID'].notna()].reset_index(drop =True)
MDM_df.rename(columns={'Eddress Product ID':'Eddress Product Id'}, inplace=True)


MDM_Status_sync_format = pd.merge(MDM_df, 
                      New_status_df, 
                      on ='Eddress Product Id', 
                      how ='left')

MDM_Status_sync_format = MDM_Status_sync_format\
                        [['Action','Type','ID','Name','Buying Categories','GS1 Taxonomy','GIPID',\
                        'Trade Item Identification: GTIN','Colli','Netsuite Vendor ID',\
                        'Eddress Product Id','Segment_New']]

MDM_Status_sync_format = MDM_Status_sync_format[MDM_Status_sync_format['Segment_New'].\
                        notna()].reset_index(drop = True)

#-------------------------------
#Create a ouptut to excel
MDM_Status_sync_format.to_excel("/Users/swaroopbasavarajmusti/Desktop/MDM_Status_sync_format.xlsx")